In [1]:
import torch
import numpy

from torch import tensor
from torch import nn
from torch import optim
from torch.autograd import Variable

from numpy import array

# Parameters

In [2]:
# Number of recognized words you put in input
nb_input = 3

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 3

# RNN implementation

In [21]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        self.layer = nn.Sequential(
            nn.RNN(nb_inputs, nb_layers, nb_neurons, nb_outputs, dropout=0.5, batch_first=True),
            nn.Softmax(nb_outputs)
        )
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, inputs):
        x = self.layer(inputs)
        return x
    
def learn(rnn, batch_list, num_epochs=1,):
    rnn.train()
    losses = []
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, target) in enumerate(batch_list):
            data, target = Variable(data), Variable(target)
            
            
            output = rnn(data)
            loss = criterion(output, target)

            losses.append(loss.data.item())

            rnn.optimizer.zero_grad()

            loss.backward()
            rnn.optimizer.step()

            if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
                print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                        epoch, 
                        batch_idx * len(data), 
                        len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), 
                        loss.data.item()), 
                        end='')

# Let's try our RNN

In [22]:
# Basically, meet a 1 and activate the first output; the second one if only 0 have been encountered
batch_list = [
    (tensor([ [0, 0], [0, 0], [0, 0] ]), tensor([0, 1])),
    (tensor([ [0, 0], [0, 0], [0, 0], [0, 0] ]), tensor([0, 1])),
    (tensor([ [1, 0], [0, 0], [0, 0] ]), tensor([1, 0])),
    (tensor([ [0, 0], [1, 0], [0, 0] ]), tensor([1, 0]))
]

rnn = RNN(2, 3, 2, 2, 0.01)

learn(rnn, batch_list)

RuntimeError: input must have 3 dimensions, got 2